# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

### Preprocesamiento

Problemas a resolver en el preprocesamiento de los datos:
* Informacion desconocida (unknown, Unknown, NaN)
* Entradas categóricas nominales y ordinales

In [ ]:
import pandas
import numpy
import sklearn
import warnings
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from feature_engine.imputation import CategoricalImputer
from sklearn.model_selection import train_test_split


warnings.filterwarnings('ignore')

In [ ]:
accidentes = pandas.read_csv('accidentes.csv')
accidentes.head()

In [ ]:
## Distribución de nulos
accidentes.replace(["Unknown","unknown"], numpy.NaN, inplace=True)
accidentes.isnull().sum()

In [ ]:
## Las entradas que tengan menos de 500 nulos se ignorarán ya que se tiene suficientes datos 
## menos sexo que es facil de inputarle valor.

accidentes.dropna(subset=
        ['medians',
         'surface',
         'weather',
         'collision',
         'car_movement',
         'acc_cause'],
        inplace=True)

accidentes.isnull().sum()

In [ ]:
# División del conjunto de datos: 60% train, 20% test, 20% validation

train, not_train = train_test_split(accidentes, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

In [ ]:
## categorias ordinales -> OrdinalEncoder
## categorias nominales -> OneHotEncoder, Vectores, BinaryEncoder, HashEncoder

## one hot enconder y rellenar los nulos con sklearn

age_encoder = OrdinalEncoder()
sex_encoder = OrdinalEncoder()


mapeador1 = DataFrameMapper( ## mejorar imputers, iterative imputer, knn imputer
            [
                (['age'],[CategoricalImputer(imputation_method='frequent'), age_encoder]),
                (['sex'],[CategoricalImputer(imputation_method='frequent'), sex_encoder]),
                (['exp'],[CategoricalImputer(imputation_method='frequent'), OneHotEncoder()]),
                (['medians'],[OneHotEncoder()]),
                (['junction'],[CategoricalImputer(imputation_method='frequent'), OneHotEncoder()]),
                (['surface'],[OneHotEncoder()]),
                (['light'],[OneHotEncoder()]),
                (['weather'],[OneHotEncoder()]),
                (['collision'],[OneHotEncoder()]),
                (['car_movement'],[OneHotEncoder()]),
                (['acc_cause'],[OneHotEncoder()]),
            ])

mapeador1.fit(train)

muestra = train.sample(1)

mapeador1.transform(muestra)

In [ ]:
mapeador1.transformed_names_

In [ ]:
## Cuales son las categorias de ordinales?

age_encoder.categories_ ## ['18-30': 0, '31-50': 1, 'Over 51': 2, 'Under 18': 3]

In [ ]:
sex_encoder.categories_ ## ['Female' : 0, 'Male' : 1]

# K vecinos más cercanos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [ ]:
K = 3 ## hiperparámetro a experimentar

knn = KNeighborsClassifier(n_neighbors=K)

tuberia_knn = Pipeline([
    ('mapper', mapeador1),
    ('classifier', knn),
])

tuberia_knn.fit(train, train.severity)
prediccion_knn = tuberia_knn.predict(validation)

prediccion_knn

df = pandas.DataFrame(prediccion_knn, columns=['severity'])
df.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(validation.severity, prediccion_knn, target_names=['fatal', 'grave', 'leve']))

# Árboles de decisión

In [ ]:
from sklearn import tree

In [ ]:
arbol = tree.DecisionTreeClassifier(max_depth=50) ## Ver hiperparametros

tuberia_arbol = Pipeline([
    ('mapper', mapeador1),
    ('classifier', arbol),
])

tuberia_arbol.fit(train, train.severity)
prediccion_arbol = tuberia_arbol.predict(validation)

prediccion_arbol

df_arbol = pandas.DataFrame(prediccion_arbol, columns=['severity'])
df_arbol.groupby('severity').aggregate({'severity':'count'})


# Árboles potenciados por gradiente

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
boost_model = GradientBoostingClassifier()

boo_model = Pipeline([
    ('mapper', mapeador1),
    ('classifier', boost_model),
])

boo_model.fit(train, train.severity)
prediccion_boo = boo_model.predict(validation)

prediccion_boo

df_arbol = pandas.DataFrame(prediccion_boo, columns=['severity'])
df_arbol.groupby('severity').aggregate({'severity':'count'})

# Métrica

El sentido de utilizar un clasificador para los accidentes de tráfico es tratar de preever accidentes según su gravedad. Por lo tanto, la métrica **"Acuracy"** no interesa ya que ésta es indiferente a la gravedad del accidente. Más bien interesa la métrica **"Recall"** para que no se escape ningún caso de las clases, sobre todo si es fatal. No interesa tanto la métrica **"Precision"** ya que no es tan grave definir como fatal un accidente mientras no lo era.


A continuación evaluaremos la métrica seleccionada con los tres modelos seleccionados sin mejora de hiperparametros, ni ingenieria de entradas

In [ ]:
print(metrics.classification_report(validation.severity, prediccion_knn, target_names=['fatal', 'grave', 'leve']))

In [ ]:
print(metrics.classification_report(validation.severity, prediccion_arbol, target_names=['fatal', 'grave', 'leve']))

In [ ]:
print(metrics.classification_report(validation.severity, prediccion_boo, target_names=['fatal', 'grave', 'leve']))

# Ingeniería de variables de entrada

# Evaluación de modelos

# Curva de aprendizaje